<h1>Dictionary POC for Tide times</h1> 
<hr>
<p>POC - From the set of tides for a day, determine which are the highest and lowest water marks. From this, I can derive the tidal range</p>

In [10]:
import pprint
from dataclasses import dataclass
from datetime import datetime

@dataclass
class TideMark:
    t_date: int
    t_seq: int
    t_type: str  # High or Low
    t_height: float
    t_time: datetime


tide_marks = [
    TideMark(t_date='04', t_seq=0, t_type='Low', t_height='0.7', t_time='01:08'),
    TideMark(t_date='04', t_seq=1, t_type='High', t_height='3.4', t_time='07:46'),
    TideMark(t_date='04', t_seq=2, t_type='Low', t_height='0.7', t_time='13:21'),
    TideMark(t_date='04', t_seq=3, t_type='High', t_height='3.7', t_time='19:59'),
    
    TideMark(t_date='05', t_seq=0, t_type='Low', t_height='0.6', t_time='01:53'),
    TideMark(t_date='05', t_seq=1, t_type='High', t_height='6.5', t_time='08:28'),
    TideMark(t_date='05', t_seq=2, t_type='Low', t_height='0.2', t_time='14:03'),
    TideMark(t_date='05', t_seq=3, t_type='High', t_height='3.8', t_time='20:37'),
    
    TideMark(t_date='06', t_seq=0, t_type='Low', t_height='0.6', t_time='02:33'),
    TideMark(t_date='06', t_seq=1, t_type='High', t_height='3.5', t_time='09:01'),
    TideMark(t_date='06', t_seq=2, t_type='Low', t_height='0.6', t_time='14:39'),
    TideMark(t_date='06', t_seq=3, t_type='High', t_height='3.8', t_time='21:08'),
    
    TideMark(t_date='07', t_seq=0, t_type='Low', t_height='0.6', t_time='03:06'),
    TideMark(t_date='07', t_seq=1, t_type='High', t_height='3.5', t_time='09:28'),
    TideMark(t_date='07', t_seq=2, t_type='Low', t_height='0.6', t_time='15:10'),
    TideMark(t_date='07', t_seq=3, t_type='High', t_height='3.7', t_time='21:33')
]



In [11]:
# Get a sorted list of the unique dates for this set of tides. 
# Example: [4, 5, 6, 7]
unique_dates = list(set(int(mark.t_date) for mark in tide_marks))
unique_dates.sort()

# Make a dictionary: for each day, set a placeholder for the High and Low
# marks, which will later be overwritten with any sensible value for a tide
# in metres. Example of this initialisation output:
# {4: {'High': -100, 'Low': 100},
#  5: {'High': -100, 'Low': 100},
#  6: {'High': -100, 'Low': 100},
#  7: {'High': -100, 'Low': 100}}
water_marks = {}
for d in unique_dates:
    water_marks[d] = {'Low':100,'High':-100}
    
pprint.pprint(water_marks)

{4: {'High': -100, 'Low': 100},
 5: {'High': -100, 'Low': 100},
 6: {'High': -100, 'Low': 100},
 7: {'High': -100, 'Low': 100}}


In [12]:
# convert to the data class to an array. This is just for simplicity when walking the data.
# Example of test_tides:
# test_tides: [(4, 0, 'Low', 0.7, '01:08'), (4, 1, 'High', 3.4, '07:46'),...
# The water_marks dict then compares the existing value (which may be the extreme initial value
# or a "sensible" value) with the newer value. If a new high or low, then the dictionary updates.
test_tides = [(int(mark.t_date), mark.t_seq, mark.t_type, float(mark.t_height), mark.t_time) for mark in tide_marks]

for test_tide in test_tides:
    test_date = test_tide[0]
    test_type = test_tide[2]
    new_height = test_tide[3]
    
    #print(water_marks[test_date])
    #print(water_marks[test_date][test_type])
    if  test_type == 'Low':
        if  new_height < water_marks[test_date][test_type]:
            water_marks[test_date][test_type] = new_height
    else: # High
        if  new_height > water_marks[test_date][test_type]:
            water_marks[test_date][test_type] = new_height
    
pprint.pprint(water_marks)
expected_values = {
    4: {'High': 3.7, 'Low': 0.7},
    5: {'High': 6.5, 'Low': 0.2},
    6: {'High': 3.8, 'Low': 0.6},
    7: {'High': 3.7, 'Low': 0.6}
}

assert(expected_values == water_marks)


{4: {'High': 3.7, 'Low': 0.7},
 5: {'High': 6.5, 'Low': 0.2},
 6: {'High': 3.8, 'Low': 0.6},
 7: {'High': 3.7, 'Low': 0.6}}


In [13]:
# get the tidal range
for i in water_marks:
    t_high = water_marks[i]['High']
    t_low = water_marks[i]['Low']
    water_marks[i]['TidalRange'] = round(t_high - t_low,2)
pprint.pprint(water_marks)

{4: {'High': 3.7, 'Low': 0.7, 'TidalRange': 3.0},
 5: {'High': 6.5, 'Low': 0.2, 'TidalRange': 6.3},
 6: {'High': 3.8, 'Low': 0.6, 'TidalRange': 3.2},
 7: {'High': 3.7, 'Low': 0.6, 'TidalRange': 3.1}}


In [ ]:
# Now that the objects are populated, we can loop over these to
# create the csv for importing into a spreadsheet

